# projectGoal

currently ts camera returns range, bearing, and yaw of how it sees the apriltag on the goal<br>
problem is we're tryna score the ball not shoot it at a glorified qr code<br>
the objective here is to do some trig to create a method that takes how the camera sees the apriltag and translate the start pos from camera to the shooter and the end pos from the apriltag to the goal<br>
we're gonna need range, bearing, elevation, and **yaw** cuz the apriltag angle affects ts end offset not the camera angle<br>
we're gonna do the math in Python and then port it to Java like a real one<br>

first step is converting ts deg to rad cuz np like that<br>
second step is turn ts into XYZ translation

In [1]:
import numpy as np

def projectGoal(rDist, bDeg, eDeg, yDeg):
    initBRad = np.radians(bDeg)
    initERad = np.radians(eDeg)
    initYRad = np.radians(yDeg)

    initXYDist = rDist * np.cos(initERad)
    initZDist = rDist * np.sin(initERad)
    initXDist = initXYDist * np.sin(initBRad)
    initYDist = initXYDist * np.cos(initBRad)

    print(initXDist)
    print(initYDist)
    print(initZDist)

In [8]:
projectGoal(67, 67, 67, 67)

24.097883311344813
10.22894458962359
61.673825181313504


In [5]:
print((24.097883311344813 ** 2 + 10.22894458962359 ** 2 + 61.673825181313504 ** 2) ** 0.5)

67.0


W ts works now js convert the extake and goal offsets into XYZ translations also<br>
lets just make all that range bearing deg to XYZ a function<br>
and lets also made an inverse of that function cuz itll come in handy ltr

In [10]:
import numpy as np

def projectGoal(rDist, bDeg, eDeg, yDeg):
    pass

def traceRBE(rDist, bRad, eRad):
    distXY = rDist * np.cos(eRad)
    distZ = rDist * np.sin(eRad)
    distX = distXY * np.sin(bRad)
    distY = distXY * np.cos(bRad)

    return distX, distY, distZ

def traceXYZ(x, y, z):
    rDist = np.sqrt(x*x + y*y + z*z)
    eRad = np.arctan2(z, np.sqrt(x*x + y*y))
    bRad = np.arctan2(x, y)

    return rDist, bRad, eRad

In [14]:
xyz = traceRBE(67, np.radians(67), np.radians(67))
print(xyz)
print(traceXYZ(*xyz))

(np.float64(24.097883311344813), np.float64(10.22894458962359), np.float64(61.673825181313504))
(np.float64(67.0), np.float64(1.1693705988362009), np.float64(1.1693705988362009))


W ok so the maths are mathing now lets bring ts all together<br>
we take in an AprilTagDetection range bearing elevation and yaw, convert it to XYZ. get the constant goal offset, convert to RBE, rotate by yaw, convert it back to XYZ. add both translations and convert back to RBE.

In [ ]:
def projectGoal(rDist, bDeg, eDeg, yDeg):
    initXYZ = traceRBE(67, np.radians(67), np.radians(67))
    goalRBE = traceXYZ(0.740128657074, 12.9696232669, 9.25) # real offset that will be used in prod
    

### Resources

[how apriltags return data](https://ftc-docs-cdn.ftclive.org/booklets/en/april_tags.pdf)